In [2]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 51.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 71.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 60.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 114.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 41.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

In [4]:
data = pd.read_csv('train_final.csv')
data.head()

,id,img,label,text
0,53924,img/53924.png,0,mom : can you mash the vegies me :
1,87619,img/87619.png,0,tap and say amen! if jesus has blessed you today
2,84125,img/84125.png,0,when you beat the red light
3,63029,img/63029.png,0,someone collect this white trash
4,26903,img/26903.png,0,"i don't have ex's. i have y's. like, ""y the he..."


In [5]:
data['label'].value_counts()


0    3000
1    3000
Name: label, dtype: int64

In [7]:
data_test = pd.read_csv('test_final.csv')
data_test.head()

,id,img,label,text
0,8291,img/08291.png,1,white people is this a shooting range
1,46971,img/46971.png,1,bravery at its finest
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...
4,80243,img/80243.png,1,mississippi wind chime


In [8]:
data_test['label'].value_counts()


1    250
0    250
Name: label, dtype: int64

In [9]:
class_mapping = {
    0:'non_hate_full',
    1:'hate_full'
}

In [10]:
data['label'] = data['label'].map(class_mapping)
data_test['label'] = data_test['label'].map(class_mapping)

In [11]:
# final_data = pd.concat([data,data_test],axis=0)
# final_data.head()

In [12]:
X_train = data.text.tolist()
X_test = data_test.text.tolist()

y_train = data['label'].tolist()
y_test = data_test['label'].tolist()

# data = final_df.append(final_df_test, ignore_index=True)

class_names = ["non_hate_full","hate_full"]


print('size of training set: %s' % (len(data['text'])))
print('size of validation set: %s' % (len(data_test['text'])))
print(data['label'].value_counts())

size of training set: 6000
size of validation set: 500
non_hate_full    3000
hate_full        3000
Name: label, dtype: int64


In [13]:
encoding = {
    'non_hate_full':0,
    'hate_full':1
}



# Integer values for each class
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

In [14]:
class_names = ["non_hate_full","hate_full"]
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350,
                                                                       max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [15]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 350


/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


done.


In [22]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train),
                             val_data=(x_test, y_test),
                             batch_size=12)

In [23]:
learner.fit_onecycle(2e-5, 5)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
500/500 [==============================] - 513s 1s/step - loss: 0.6328 - accuracy: 0.6355 - val_loss: 0.6788 - val_accuracy: 0.6020
Epoch 2/5
500/500 [==============================] - 502s 1s/step - loss: 0.5667 - accuracy: 0.7028 - val_loss: 0.6808 - val_accuracy: 0.6000
Epoch 3/5
500/500 [==============================] - 508s 1s/step - loss: 0.5065 - accuracy: 0.7458 - val_loss: 0.7127 - val_accuracy: 0.6000
Epoch 4/5
500/500 [==============================] - 508s 1s/step - loss: 0.3964 - accuracy: 0.8092 - val_loss: 0.9400 - val_accuracy: 0.5660
Epoch 5/5
500/500 [==============================] - 508s 1s/step - loss: 0.2856 - accuracy: 0.8592 - val_loss: 1.0071 - val_accuracy: 0.5680


In [24]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

16/16 [==============================] - 19s 872ms/step
               precision    recall  f1-score   support

non_hate_full       0.56      0.66      0.61       250
    hate_full       0.58      0.47      0.52       250

     accuracy                           0.57       500
    macro avg       0.57      0.57      0.56       500
 weighted avg       0.57      0.57      0.56       500



array([[166,  84],
       [132, 118]])

In [25]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['non_hate_full', 'hate_full']

In [26]:
predictor.save('models/text_model/')